## Implementation of CNN from SCRATCH

In [1]:
import numpy as np
import math

In [169]:
class Conv2d:
    id_ = 0
    def __init__(self,input_x:np.ndarray,filter_shape:tuple,number_of_filters:int,padding:int=1,
                 stride:int=1,max_pool_shape:tuple=None,max_pool_stride:int=2):
        self.input_x = input_x # (channel,H,W)
        self.filter_shape = filter_shape # (F_h,F_W)
        self.number_of_filters = number_of_filters
        self.padding = padding
        self.stride = stride
        self.filters = None
        self.max_pool_shape = max_pool_shape
        self.max_pool_stride = max_pool_stride        
        self.id_ = Conv2d.id_ 
        Conv2d.id_ += 1
        
    def xavier_weight_init(self,shape:tuple,fan_in:int,fan_out:int) -> np.ndarray:
        return np.random.randn(*shape)*np.sqrt(2/(fan_in+fan_out))
        
    # forward
    def __call__(self,is_predicting=False) -> np.ndarray:
        # Padding
        input_x = None
        if self.padding != 0:
            ch0 = np.pad(self.input_x[0],1)
            ch1 = np.pad(self.input_x[1],1)
            ch2 = np.pad(self.input_x[2],1)
            input_x = np.vstack([[ch0],[ch1],[ch2]])
        else:
            input_x = self.input_x
  
        
        # Calculating output Dimensions
        output_h = math.floor((((self.input_x.shape[1]+2*self.padding-self.filter_shape[0]) /self.stride) + 1 ))
        output_w = math.floor((((self.input_x.shape[2]+2*self.padding-self.filter_shape[1]) /self.stride) + 1 ))
        
        # Initializing output
        output = np.zeros((self.number_of_filters,output_h,output_w)) # (number_of_filters,H,W)
        
        # Initializing filters
        if not is_predicting:
            self.filters = [self.xavier_weight_init(
                shape=(input_x.shape[0],self.filter_shape[0],self.filter_shape[1]),
                fan_in = input_x.shape[0] * input_x.shape[1] * input_x.shape[2],
                fan_out = self.filter_shape[0] * self.filter_shape[1] * input_x.shape[0] if self.max_pool_shape is None else self.filter_shape[0] * self.filter_shape[1] * input_x.shape[0] /(self.max_pool_shape[0]*self.max_pool_shape[1])  
            ) for _ in range(self.number_of_filters)]

        # Convolving
        for index, filter_ in enumerate(self.filters):
            out = np.empty((output_h,output_w))
            row_counter = 0
            col_counter = 0
            for row in np.arange(0,input_x.shape[1],step=self.stride):
                if row+self.filter_shape[0] > input_x.shape[1] or row > input_x.shape[1]:
                    pass
                else:
                    for col in np.arange(0,input_x.shape[2],step=self.stride):
                        if col+self.filter_shape[1] >  input_x.shape[2] or col > input_x.shape[2]:
                            pass
                        else:
                            out[row_counter,col_counter] =(input_x[:,row:row+self.filter_shape[0],col:col+self.filter_shape[1]] * filter_).sum()
                            col_counter += 1
                    
                    col_counter = 0
                    row_counter += 1
            output[index] = out
        
        
        # Max Pooling       
        if self.max_pool_shape is not None:
            pooling_output = np.zeros((output.shape[0],
                                       ((output.shape[1]-self.max_pool_shape[0])//self.max_pool_stride)+1,
                                       ((output.shape[2]-self.max_pool_shape[1])//self.max_pool_stride)+1))
            for i in range(output.shape[0]):
                col_counter = 0
                row_counter = 0
                for row in np.arange(0,input_x.shape[1],step=self.max_pool_stride):
                    if row+self.max_pool_shape[0] > output.shape[1]:
                        pass
                    else:
                        for col in np.arange(0,input_x.shape[2],step=self.max_pool_stride):
                            if col+self.max_pool_shape[1] > output.shape[2]:
                                pass
                            else:
                                pooling_output[i,row_counter,col_counter] = output[i,row:row+self.max_pool_shape[0],col:col+self.max_pool_shape[1]].max()
                                col_counter += 1 
                        row_counter += 1
                        col_counter = 0
       
            return pooling_output
        
        return output
    
#     def backward(self):
        
        
        
        
        

In [170]:
A = np.arange(75).reshape((3,5,5))

In [171]:
A

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]],

       [[25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49]],

       [[50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64],
        [65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74]]])

In [172]:
conv2d = Conv2d(input_x=A,filter_shape=(2,2),number_of_filters=5,padding=1,stride=2,max_pool_shape=(2,2))

In [173]:
conv2d(is_predicting=False).shape

(5, 1, 1)